In [ ]:
# parse_dates参数：
# 将csv中的时间字符串转换成日期格式
# 地点在建筑物还是十字路口对预测犯罪类型有助

In [ ]:
import pandas as pd
from shapely.geometry import  Point
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
from matplotlib import cm
import urllib.request
import shutil
import zipfile
import os
import re
import contextily as ctx
import geoplot as gplt
import lightgbm as lgb
import eli5
from eli5.sklearn import PermutationImportance
from lightgbm import LGBMClassifier
from matplotlib import pyplot as plt
from pdpbox import pdp, get_dataset, info_plots
import shap

In [ ]:
train = pd.read_csv('../input/train.csv', parse_dates=['Dates'])
#对Dates列名对应的时间转换格式   parse_dates参数： 将csv中的时间字符串转换成日期格式 
test = pd.read_csv('../input/test.csv', parse_dates=['Dates'], index_col='Id')
#指定 Id 列作为索引
# 在index_col指定表格中的第几列作为Index时需要小心。如本例中，指定参数index_col=0，
# 则此时会以新生成的time_date列而不是name作为Index。因此保险的方法是指定列名，如index_col = 'name'


In [ ]:
print('First date: ', str(train.Dates.describe()['first']))
print('Last date: ', str(train.Dates.describe()['last']))
print('Test data shape ', train.shape)

In [ ]:
train.head()

In [ ]:
train.dtypes  #观察各属性 数据类型
#The dataset contains a lot of 'object' variables (aka strings) that we will need to encode.

In [ ]:
train.duplicated().sum()
#它还包含2323个重复项，我们应将其删除。
#我们还将使用坐标评估数据点的位置。

In [ ]:
#Detailed TODO
#画出图像，以地理图为画布，DataFrame画出犯罪地点的经纬度
def create_gdf(df):
    gdf = df.copy()
    gdf['Coordinates'] = list(zip(gdf.X, gdf.Y))
    gdf.Coordinates = gdf.Coordinates.apply(Point)
    gdf = gpd.GeoDataFrame(
        gdf, geometry='Coordinates', crs={'init': 'epsg:4326'})
    return gdf

train_gdf = create_gdf(train)

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
ax = world.plot(color='white', edgecolor='black')
train_gdf.plot(ax=ax, color='red')
plt.show()
#图像显示海区是犯罪地点说明这些数据是错误无用的

In [ ]:
print(train_gdf.loc[train_gdf.Y > 50].count()[0])
train_gdf.loc[train_gdf.Y > 50].sample(5)
#错误数据输出：67个
#取五个样本瞧一瞧
#发现 经纬度对应120 和 90 

In [ ]:
#从异常值和重复项中清除数据集
train.drop_duplicates(inplace=True)
#把经纬度对应120 和 90 的数据清除
train.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)
test.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)
#inplace=True就把{'X': -120.5, 'Y': 90.0}替换成np.NaN

imp = SimpleImputer(strategy='mean')
#数据缺失值补全方法 sklearn.impute.SimpleImputer
#默认属性 missing_values=np.nan  

#imp.fit_transform(df) 填充
for district in train['PdDistrict'].unique():
    #train['PdDistrict'] == district 返回相应的索引列表
    #e.g. train.loc[['Id1','Id2']]--这些索引对应的dataframe
        train.loc[train['PdDistrict'] == district, ['X', 'Y']] = imp.fit_transform(
            train.loc[train['PdDistrict'] == district, ['X', 'Y']])
        test.loc[test['PdDistrict'] == district, ['X', 'Y']] = imp.transform(
        test.loc[test['PdDistrict'] == district, ['X', 'Y']])
##Detailed TODO  imp.fit_transform
train_gdf = create_gdf(train)

In [ ]:
#我们检查变量
#画图观察各属性中哪些是对预测有用的
#Dates Hours
#日期和星期几
# 这些变量在2003年1月1日至2015年5月13日（以及周一至周日）之间均匀分布，并在训练和测试数据集之间进行分配（如前所述）。我们没有注意到这些变量有任何异常。
# 事故的中位数频率为每天389次，标准偏差为48.51。

#画出事件每天发生几起的分布---发现像是个正态分布，平平无奇
#Detailed TODO 
col = sns.color_palette()

train['Date'] = train.Dates.dt.date
train['Hour'] = train.Dates.dt.hour

plt.figure(figsize=(10, 6))
data = train.groupby('Date').count().iloc[:, 0]
sns.kdeplot(data=data, shade=True)
plt.axvline(x=data.median(), ymax=0.95, linestyle='--', color=col[1])
plt.annotate(
    'Median: ' + str(data.median()),
    xy=(data.median(), 0.004),
    xytext=(200, 0.005),
    arrowprops=dict(arrowstyle='->', color=col[1], shrinkB=10))
plt.title(
    'Distribution of number of incidents per day', fontdict={'fontsize': 16})
plt.xlabel('Incidents')
plt.ylabel('Density')
plt.legend().remove()
plt.show()

In [ ]:
#同样，整个星期的事件发生频率也没有明显的偏差。因此，我们预计该变量不会在预测中发挥重要作用。
#画出事件相对于星期几的直方图，发现分布均匀
#Detailed TODO 
data = data.reindex([
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday',
    'Sunday'
])

plt.figure(figsize=(10, 5))
with sns.axes_style("whitegrid"):
    ax = sns.barplot(
        data.index, (data.values / data.values.sum()) * 100,
        orient='v',
        palette=cm.ScalarMappable(cmap='Reds').to_rgba(data.values))

plt.title('Incidents per Weekday', fontdict={'fontsize': 16})
plt.xlabel('Weekday')
plt.ylabel('Incidents (%)')

plt.show()

In [ ]:
# 类别
#警察局共记录了39个离散类别的事件，最常见的是盗窃/盗窃（19.91％），非/犯罪（10.50％）和殴打（8.77％）。
#竖轴对应类别---画出直方图，横轴是发生率
data = train.groupby('Category').count().iloc[:, 0].sort_values(
    ascending=False)
data = data.reindex(np.append(np.delete(data.index, 1), 'OTHER OFFENSES'))

plt.figure(figsize=(10, 10))
with sns.axes_style("whitegrid"):
    ax = sns.barplot(
        (data.values / data.values.sum()) * 100,
        data.index,
        orient='h',
        palette="Reds_r")

plt.title('Incidents per Crime Category', fontdict={'fontsize': 16})
plt.xlabel('Incidents (%)')

plt.show()

In [ ]:
# 警察区
# 城市的不同地区之间存在显着差异，其中南部地区的事件发生率最高（17.87％），其次是Mission（13.67％）和Northern（12.00％）。
# Downloading the shapefile of the area 
#以洛杉矶地理图为背景画出热图，越红犯罪率越高
url = 'https://data.sfgov.org/api/geospatial/wkhw-cjsf?method=export&format=Shapefile'
with urllib.request.urlopen(url) as response, open('pd_data.zip', 'wb') as out_file:
    shutil.copyfileobj(response, out_file)
# Unzipping it
with zipfile.ZipFile('pd_data.zip', 'r') as zip_ref:
    zip_ref.extractall('pd_data')
# Loading to a geopandas dataframe
for filename in os.listdir('./pd_data/'):
    if re.match(".+\.shp", filename):
        pd_districts = gpd.read_file('./pd_data/'+filename)
        break
# Defining the coordinate system to longitude/latitude
pd_districts.crs={'init': 'epsg:4326'}

# Merging our train dataset with the geo-dataframe
pd_districts = pd_districts.merge(
    train.groupby('PdDistrict').count().iloc[:, [0]].rename(
        columns={'Dates': 'Incidents'}),
    how='inner',
    left_on='district',
    right_index=True,
    suffixes=('_x', '_y'))

# Transforming the coordinate system to Spherical Mercator for
# compatibility with the tiling background
pd_districts = pd_districts.to_crs({'init': 'epsg:3857'})

# Calculating the incidents per day for every district
train_days = train.groupby('Date').count().shape[0]
pd_districts['inc_per_day'] = pd_districts.Incidents/train_days

# Ploting the data
fig, ax = plt.subplots(figsize=(10, 10))
pd_districts.plot(
    column='inc_per_day',
    cmap='Reds',
    alpha=0.6,
    edgecolor='r',
    linestyle='-',
    linewidth=1,
    legend=True,
    ax=ax)

def add_basemap(ax, zoom, url='http://tile.stamen.com/terrain/tileZ/tileX/tileY.png'):
    """Function that add the tile background to the map"""
    xmin, xmax, ymin, ymax = ax.axis()
    basemap, extent = ctx.bounds2img(xmin, ymin, xmax, ymax, zoom=zoom, url=url)
    ax.imshow(basemap, extent=extent, interpolation='bilinear')
    # restore original x/y limits
    ax.axis((xmin, xmax, ymin, ymax))

# Adding the background
add_basemap(ax, zoom=11, url=ctx.sources.ST_TONER_LITE)

# Adding the name of the districts
for index in pd_districts.index:
    plt.annotate(
        pd_districts.loc[index].district,
        (pd_districts.loc[index].geometry.centroid.x,
         pd_districts.loc[index].geometry.centroid.y),
        color='#353535',
        fontsize='large',
        fontweight='heavy',
        horizontalalignment='center'
    )

ax.set_axis_off()
plt.show()

In [ ]:
# 地址
# 地址，作为文本字段，需要先进的技术才能将其用于预测。
#取而代之的是，在此项目中，我们将提取它来判断事故是否发生在道路上或建筑构件中。
# X-经度Y-纬度
# 我们已经测试了坐标在城市边界内。尽管经度不包含任何异常值，但纬度包含一些与北极相对应的90o值。
# 探索性可视化
# 根据项目的声明，我们需要根据时间和地点来预测每种犯罪的可能性。话虽如此，我们提供了两个图表来可视化这些变量的重要性。
#第一个介绍了9个随机犯罪类别的地理密度。我们可以看到，尽管大多数犯罪的重心位于城市的东北部，但每种犯罪在城市其他地方的密度却不同。这是一个可靠的迹象，表明位置（坐标/警区）将成为分析和预测的重要因素。
crimes = train['Category'].unique().tolist()
crimes.remove('TREA')

pd_districts = pd_districts.to_crs({'init':'epsg:4326'})
sf_land = pd_districts.unary_union
sf_land = gpd.GeoDataFrame(gpd.GeoSeries(sf_land), crs={'init':'epsg:4326'})
sf_land = sf_land.rename(columns={0:'geometry'}).set_geometry('geometry')

fig, ax = plt.subplots(3, 3, sharex=True, sharey=True, figsize=(12,12))
for i , crime in enumerate(np.random.choice(crimes, size=9, replace=False)):
    data = train_gdf.loc[train_gdf['Category'] == crime]
    ax = fig.add_subplot(3, 3, i+1)
    gplt.kdeplot(data,
                 shade=True,
                 shade_lowest=False,
                 clip = sf_land.geometry,
                 cmap='Reds',
                 ax=ax)
    gplt.polyplot(sf_land, ax=ax)
    ax.set_title(crime) 
plt.suptitle('Geographic Density of Different Crimes')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
# 第二张图显示了五种犯罪类别每小时的平均事件数。
# 显然，不同的犯罪在一天中的不同时间发生的频率不同。
# 例如，卖淫在傍晚和整个晚上都有，赌博事件从深夜开始直到早晨，而盗窃案则从清晨到下午进行。
# 像以前一样，这些都是清晰的证据，表明时间参数也将发挥重要作用。
# 画图横轴为0-24h 为不同犯罪类型做曲线，竖轴为犯罪数
data = train.groupby(['Hour', 'Date', 'Category'],
                     as_index=False).count().iloc[:, :4]
data.rename(columns={'Dates': 'Incidents'}, inplace=True)
data = data.groupby(['Hour', 'Category'], as_index=False).mean()
data = data.loc[data['Category'].isin(
    ['ROBBERY', 'GAMBLING', 'BURGLARY', 'ARSON', 'PROSTITUTION'])]

sns.set_style("whitegrid")
fig, ax = plt.subplots(figsize=(14, 4))
ax = sns.lineplot(x='Hour', y='Incidents', data=data, hue='Category')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=6)
plt.suptitle('Average number of incidents per hour')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
# 算法与技术
# 特定问题是典型的多类分类问题，有几种算法可以解决该问题。
# 最初，我们使用线性模型（随机梯度下降），最近邻（K个最近邻），集成方法（随机森林和AdaBoost）和增强算法（XGBoost和LIghtGBM）中的几种合适算法进行了评估，
# 并使用默认参数进行评估是否其中任何一个有重大的领先优势
# 其实SGD效果最好
#我们决定使用LightGBM，因为它具有超参数调整的效率和多功能性。 
#LightGBM是一种决策树增强算法，它使用基于直方图的算法，该算法将连续特征（属性）值存储到离散的bin中。此技术可加快训练速度并减少内存使用量。用外行术语来说，该算法的工作方式如下：
# 1.使决策树拟合数据
# 2.评估模型
# 3.给不正确的样本增加权重。
# 4.选择具有最大增量损失的叶子进行生长。
# 5.Grow the tree。
# 6.TO 2 评估模型

# 我们需要设置两种基准。第一个将是naive预测。此预测将是基线得分，可与我们模型的得分进行比较，以评估我们是否有任何重大进展。
# 在多类分类中，计算基线的最佳方法是假设每个类别的概率等于其在训练集中的平均频率。
# 通过将每个类别的事件总和除以训练集的行数，可以轻松地计算频率。
# 39类犯罪   每类预测的发生率 为数据集的频率

naive_vals = train.groupby('Category').count().iloc[:,0]/train.shape[0]
n_rows = test.shape[0]

submission = pd.DataFrame(
    np.repeat(np.array(naive_vals), n_rows).reshape(39, n_rows).transpose(),
    columns=naive_vals.index)


In [ ]:
# 用这种方法计算的基线为2.68015。 #https://www.kaggle.com/yannisp/sf-crime-analysis-prediction-naive-prediction
# 我们可以注意到，该基线已经低于分类器的初始得分。
# 另一个至关重要的基准通常是“人类性能”，可以作为贝叶斯错误率的替代指标。
# 特定问题不属于人类擅长的领域（例如计算机视觉或NLP），
# 因此，作为贝叶斯错误率的代理，我们将使用到目前为止最佳内核的分数，这是用户谢尔盖·列别捷夫（Sergey Lebedev）调整的，初始基准得分2.29318。
# 基线得分和贝叶斯错误率之间的距离很小，这表明这是一个难题，改善幅度很小。

#数据处理
#按照问题陈述中描述的方法，我们确定了2323个重复值和67个错误的纬度。
#删除重复项并估算离群值。
#然后，我们创建了特征：
#从“日期”字段中，我们提取了日期，月份，年份，小时，分钟，工作日以及从第一天开始的天数。
#我们从“地址”字段中提取了事件是否发生在十字路口或建筑构件上。
def feature_engineering(data):
    data['Date'] = pd.to_datetime(data['Dates'].dt.date)
    data['n_days'] = (
        data['Date'] - data['Date'].min()).apply(lambda x: x.days)
    #从第一天开始的天数
    data['Day'] = data['Dates'].dt.day
    data['DayOfWeek'] = data['Dates'].dt.weekday
    #工作日
    data['Month'] = data['Dates'].dt.month
    data['Year'] = data['Dates'].dt.year
    data['Hour'] = data['Dates'].dt.hour
    data['Minute'] = data['Dates'].dt.minute
    data['Block'] = data['Address'].str.contains('block', case=False)
    #“地址”字段中提取了事件是否发生在十字路口或建筑构件上---bool---True--False
    data.drop(columns=['Dates','Date','Address'], inplace=True)
        
    return data

train = feature_engineering(train)
train.drop(columns=['Descript','Resolution'], inplace=True)
test = feature_engineering(test)
train.head()

In [ ]:
# 特征缩放
# 决定继续使用基于树的算法，因此无需对最终数据集进行缩放。
# 特征选择
# 经过上述功能设计，我们最终获得了11个特征。 时间-地区-X-Y-建筑物地区
#为了确定其中是否增加了模型的复杂性而不增加模型的可观收益，我们使用了Permutation Importance置换重要性方法。

# 删一个特征看一次损失值
# 想法是，可以通过查看功能不可用时损耗减少多少来衡量功能的重要性。
# 为此，我们可以从数据集中删除每个特征，重新训练估计量并检查影响。
# 这样做将需要重新训练每个功能的估计量，这可能需要大量计算。
# 取而代之的是，我们可以通过特征的打乱后的值将其替换为噪声。？？？
# 上述技术的实现表明，无需删除任何特征，因为所有特征都对数据集产生积极影响。

#下例：删除PdDistrict 后用LGBMClassifier模型训练后得每个功能的估计量
#即将离散型的数据转换成 0到n−1 之间的数，这里 n是一个列表的不同取值的个数
#可以认为是某个特征的所有不同取值的个数。

#编码
le1 = LabelEncoder()
train['PdDistrict'] = le1.fit_transform(train['PdDistrict'])
test['PdDistrict'] = le1.transform(test['PdDistrict'])
le2 = LabelEncoder()
y = le2.fit_transform(train.pop('Category'))

train_X, val_X, train_y, val_y = train_test_split(train, y)

model =LGBMClassifier(objective='multiclass', num_class=39).fit(train_X, train_y)

perm = PermutationImportance(model).fit(val_X, val_y)
eli5.show_weights(perm, feature_names=val_X.columns.tolist())


In [ ]:
# 建立初始模型
# 为了建立模型，我们使用了LightGBM的Python API。首先，我们通过组合特征和目标形成数据集，并将PdDistrict声明为分类变量来使用'lightgbm.Dataset（）。
# 然后，我们将交叉验证与提前停止（10个回合）和参数一起使用：
# Objective = 'multiclass',
# 'Metric = ‘multi_logloss',
# 'Num_class = 39
#上面的设置在23个epoch后获得了2.46799的交叉验证得分，在测试集上获得了2.49136。


#作为最终模型的结论，我们对100个时代使用了五重交叉验证，并通过贝叶斯优化提前停止。
#此外，我们创建了一个自定义的回调函数，以便我们可以编写可由Tensorboard读取的正确日志。这样，我们可以实时监控验证过程。
#以上在某种程度上是一个迭代过程。
#我们运行优化过程，直到在Tensorboard中注意到模型收敛为止。然后，我们停止并评估了结果，然后移至下一个迭代。 （此处为流程示例）
# 1.优化超参数
# 2.模型收敛后进行下一轮微调  比上次多调了两个超参数

In [ ]:
#最终模型
#See here

#LoadData

train = pd.read_csv('../input/train.csv', parse_dates=['Dates'])
test = pd.read_csv('../input/test.csv', parse_dates=['Dates'], index_col='Id')

# Data cleaning
train.drop_duplicates(inplace=True)
train.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)
test.replace({'X': -120.5, 'Y': 90.0}, np.NaN, inplace=True)

imp = SimpleImputer(strategy='mean')

for district in train['PdDistrict'].unique():
    train.loc[train['PdDistrict'] == district, ['X', 'Y']] = imp.fit_transform(
        train.loc[train['PdDistrict'] == district, ['X', 'Y']])
    test.loc[test['PdDistrict'] == district, ['X', 'Y']] = imp.transform(
        test.loc[test['PdDistrict'] == district, ['X', 'Y']])
train_data = lgb.Dataset(
    train, label=y, categorical_feature=['PdDistrict'], free_raw_data=False)

# Feature Engineering
train = feature_engineering(train)
train.drop(columns=['Descript','Resolution'], inplace=True)
test = feature_engineering(test)

# Encoding the Categorical Variables
le1 = LabelEncoder()
train['PdDistrict'] = le1.fit_transform(train['PdDistrict'])
test['PdDistrict'] = le1.transform(test['PdDistrict'])

le2 = LabelEncoder()
X = train.drop(columns=['Category'])
y= le2.fit_transform(train['Category'])

# Creating the model---模型类---import lightgbm as lgb
train_data = lgb.Dataset(
    X, label=y, categorical_feature=['PdDistrict'])

params = {'boosting':'gbdt',
          'objective':'multiclass',
          'num_class':39,
          'max_delta_step':0.9,
          'min_data_in_leaf': 21,
          'learning_rate': 0.4,
          'max_bin': 465,
          'num_leaves': 41
         }

bst = lgb.train(params, train_data, 100)

predictions = bst.predict(test)

# Submitting the results
submission = pd.DataFrame(
    predictions,
    columns=le2.inverse_transform(np.linspace(0, 38, 39, dtype='int16')),
    index=test.index)
submission.to_csv(
    'LGBM_final.csv', index_label='Id')


#2.25697

In [ ]:
#其实一个特征到底对模型影响如何呢

#hour对赌博就没有影响 对别的犯罪类型就有点影响
#from lightgbm import LGBMClassifier  ----分类器LGBMClassifier 
# Partial Dependencies 部份依赖图，39张图 横轴为hour 纵轴围殴hour得分布
model = LGBMClassifier(**params).fit(X, y, categorical_feature=['PdDistrict'])

pdp_Pd = pdp.pdp_isolate(
    model=model,
    dataset=X,
    model_features=X.columns.tolist(),
    feature='Hour',
    n_jobs=-1)

pdp.pdp_plot(
    pdp_Pd,
    'Hour',
    ncols=3)
plt.show()